In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets
import re
import scipy.interpolate
import matplotlib
import sys
import pathlib 

import invplot

In [6]:
invFile = r"\\isgs-sinkhole\geophysics\2DResistivityProjects\ChampaignCo\Fisher2022\Inversions\700E_pyEdit_topo.INV"
invFilePath = pathlib.Path(invFile)
keyPointsdict = invplot.ingest_inv(invFile, verbose=False)
list(keyPointsdict.keys())

['noIterations',
 'resistDF',
 'noDataPoints',
 'iterationDF',
 'noModelBlocks',
 'layerRow',
 'dataList',
 'refResistRow',
 'layerDepths',
 'shiftMatrixRow',
 'topoDataRow',
 'blockRow',
 'noTopoPts',
 'layerRowList',
 'noLayerRow',
 'electrodeCoordsRow',
 'fileHeader',
 'iterationStartRowList',
 'errorDistRow',
 'calcResistivityRowList',
 'fileHeaderDict',
 'noPoints',
 'iterationsInfoRow',
 'sensAndUncertainRow',
 'layerInfoRow']

## Keep Working from here down, moving these into invplot.py

Original notebook here:
https://colab.research.google.com/drive/11zkZCyaxFWivLHSYbqEznKrUkjpj31nh#scrollTo=dFZihIPUu9zW

In [ ]:
def iterationDropfun():
  iterDropDown = widgets.Dropdown(options=list(np.linspace(1, noIterations, noIterations)),
                                  description='Iteration: ',
                                  disabled=False)

  iterationPrint = int(1)
  iterationInd = iterationPrint-1
  def iter_dropdown_handler(change):
      global iterationPrint
      global iterationInd
      global titleText
      print('iterationPrint '+str(change.new))
      iterationPrint = int(change.new)  
      iterationInd = iterationPrint-1
      titleText = invFilePath.stem +'\n Iteration: '+ str(iterationPrint) +' (' +str(round(iterationDF.loc[iterationInd,'%AbsError'],1))+'% Abs Error)'
  iterDropDown.observe(iter_dropdown_handler, names='value')
  display(iterDropDown)
  print('Default: Iteration 1')
  return iterationPrint, iterationInd
iterationPrint, iterationInd = iterationDropfun()

In [ ]:
#Dataframes to make

#Input Data
def readInputData(file=InvFile,noDataPoints=noDataPoints,startRow=9):
  import csv
  with open(InvFile) as datafile: 
      filereader = csv.reader(datafile)
      start = 0
      inDataList = []
      for row in enumerate(filereader):
        if row[0] < startRow:
          continue
        elif row[0] < startRow+noDataPoints:
          inDataList.append(re.sub('\s+',' ',row[1][0]).split(' '))
        else:
          break
  inDF = pd.DataFrame(inDataList)
  if startRow == 9:
    inDF.drop([0],inplace=True,axis=1)
  inDF.astype(np.float64)
  inDF.columns=['NoElectrodes', 'A(x)', 'A(z)', 'B(x)', 'B(z)', 'M(x)', 'M(z)', 'N(x)', 'N(z)', 'Data']
  return inDF

inputDF = readInputData(file=InvFile,noDataPoints=noDataPoints,startRow=9)
calcModelDF = readInputData(startRow=calcResistivityRowList[iterationInd]+1)

def readRestOfData(InvFile=InvFile, inputDF=inputDF, layerRowList=layerRowList, noPoints=noPoints,layerDepths=layerDepths,shiftMatrixRow=shiftMatrixRow,electrodeCoordsRow=electrodeCoordsRow,topoDataRow=topoDataRow,noTopoPts=noTopoPts,sensAndUncertainRow=sensAndUncertainRow):
  #Get Electrodes
  electrodes = inputDF['A(x)'].append(inputDF['B(x)'])
  electrodes = electrodes.append(inputDF['M(x)'])
  electrodes = electrodes.append(inputDF['N(x)'])
  electrodes = electrodes.unique()
  electrodes

  #ElectrodeCoordinates
  noModelElects = shiftMatrixRow-electrodeCoordsRow-1
  electrodeCoordsDF = pd.read_table(InvFile,skiprows=electrodeCoordsRow,nrows=noModelElects, sep='\s+')
  electrodeCoordsDF.dropna(axis=1,inplace=True)
  electrodeCoordsDF.columns=['xDist','RelElevation']
  electrodeCoordsDF['ElectrodeNo'] = electrodeCoordsDF.index+1
  electrodeCoordsDF

  #Topographical Data
  topoDF = pd.read_table(InvFile,skiprows=topoDataRow+2,nrows=noTopoPts, sep='\s+')
  topoDF.reset_index(inplace=True)
  topoDF.columns=['xDist','Elevation']
  topoDF['ElectrodeNo'] = topoDF.index+1
  topoDF

  #Resistivity Model
  resistModelDF = pd.DataFrame()
  for r in enumerate(layerRowList[iterationInd]):
    layerDepth = layerDepths[r[0]]
    noPtsInLyr = noPoints[r[0]]
    currDF = pd.read_table(InvFile,skiprows=r[1]+1, nrows=noPtsInLyr,sep=',')
    currDF.columns=['ElectrodeNo','Data']
    currDF['z'] = layerDepth
    resistModelDF= resistModelDF.append(currDF,ignore_index=True).copy()
  noModelBlocks=resistModelDF.shape[0]
  #resistModelDF.plot('x','z',kind='scatter',c='Data',vmin=0,vmax=100,ylim=[layerDepth,0])


  #Shift Matrix
  shiftMatrixDF = pd.read_table(InvFile,skiprows=shiftMatrixRow+1,nrows=noModelElects, sep='\s+',header=None,index_col=0)
  shiftMatrixDF.dropna(axis=1,inplace=True)
  for c in shiftMatrixDF:
    shiftMatrixDF.rename(columns={c:'Layer'+str(int(c)-1)},inplace=True)
  #shiftMatrixDF

  #Sensitivity
  sensDF = pd.read_table(InvFile,skiprows=sensAndUncertainRow+3,nrows=noModelBlocks, sep='\s+',header=None,index_col=0)
  sensDF.dropna(axis=1,inplace=True)
  sensDF.reset_index(inplace=True)
  sensDF.columns=['BlockNo','Sensitivity', '%ApproxUncertainty']
  #sensDF
  return electrodes, electrodeCoordsDF, topoDF, shiftMatrixDF, sensDF
electrodes, electrodeCoordsDF, topoDF, shiftMatrixDF, sensDF = readRestOfData()

#Error Distribution
def readErrorData(file=InvFile,noDataPoints=noDataPoints,startRow=errorDistRow+1):
  import csv
  with open(InvFile) as datafile: 
      filereader = csv.reader(datafile)
      start = 0
      inDataList = []
      for row in enumerate(filereader):
        if row[0] < startRow:
          continue
        elif row[0] < startRow+noDataPoints:
          newrow = row[1]
          newrow.append(newrow[4][8:])
          newrow[4] = newrow[4][0:8]
          newrow = [x.strip() for x in newrow]
          inDataList.append(newrow)
        else:
          break
  inDF = pd.DataFrame(inDataList)
  return inDF
errDistDF = readErrorData()

def avgErr(errDistDF=errDistDF):
  colList=['xDist?','nFactor?','Measure1','Measure2','PercentError','MoreStacks','AvgMeasure']
  for i in range(0,5):
    errDistDF[i]=errDistDF[i].astype(np.float64)
    errDistDF.rename(columns={i:colList[i]},inplace=True)
  errDistDF['AvgMeasure'] = (errDistDF['Measure1']+errDistDF['Measure2'])/2
  return errDistDF
errDistDf = avgErr()

def mapDiff(xIn, x1,x2,y1,y2):
    if x1==xIn:
        yOut=y1
    elif x2==xIn:
        yOut = y2
    else:
        totXDiff = x2-x1
        percXDiff = (xIn-x1)/totXDiff
        totYDiff = y2-y1
        yOut = y1 + totYDiff*percXDiff
    return yOut

In [ ]:
resistModelDF = pd.DataFrame()
def getResistivityModel(InvFile=InvFile,layerRowList=layerRowList,iterationInd=iterationInd,layerDepths=layerDepths,noPoints=noPoints, electrodeCoordsDF=electrodeCoordsDF, resistModelDF=resistModelDF):
  #global electrodeCoordsDF
  global topoDF

  for r in enumerate(layerRowList[iterationInd]):
    layerDepth = layerDepths[r[0]]
    noPtsInLyr = noPoints[r[0]]
    currDF = pd.read_table(InvFile,skiprows=r[1]+1, nrows=noPtsInLyr,sep=',')
    currDF.iloc[currDF.shape[0]-1,1] =  currDF.iloc[currDF.shape[0]-1,0]
    currDF.iloc[currDF.shape[0]-1,0] = currDF.iloc[currDF.shape[0]-2,0]+1
    currDF.columns=['ElectrodeNo','Data']
    currDF['zDepth'] = layerDepth

    for i in currDF.index:
      lowerElecNo = currDF.loc[i,'ElectrodeNo']#-1
      elecInd = electrodeCoordsDF.loc[electrodeCoordsDF['ElectrodeNo']==lowerElecNo].index.values[0]
      currDF.loc[i,'x'] = (electrodeCoordsDF.loc[elecInd,'xDist'] + electrodeCoordsDF.loc[elecInd+1,'xDist'])/2
      for xT in enumerate(topoDF['xDist']):
        if xT[1] < currDF.loc[i,'x']:
          continue
        else:
          topoX1 = topoDF.loc[xT[0]-1,'xDist']
          topoX2 = topoDF.loc[xT[0],'xDist']
          topoZ1 = topoDF.loc[xT[0]-1,'Elevation']
          topoZ2 = topoDF.loc[xT[0],'Elevation']
        break
      currDF.loc[i,'zElev'] = mapDiff(currDF.loc[i,'x'],topoX1, topoX2, topoZ1, topoZ2)-currDF.loc[i,'zDepth']
    if r[0] == 0:
      surfDF = currDF.copy()
      surfDF['zElev'] = surfDF.loc[:,'zElev']+surfDF.loc[:,'zDepth']
      surfDF['zDepth'] = 0
      resistModelDF= resistModelDF.append(surfDF,ignore_index=True).copy()
    resistModelDF= resistModelDF.append(currDF,ignore_index=True).copy()
  #noModelBlocks=resistModelDF.shape[0]
  return resistModelDF

resistModelDF = getResistivityModel()

In [ ]:
def plotPretty(x,z,v,im,cbarTicks,fig,ax,colMap='jet',cMin=resistModelDF['Data'].min(),cMax=resistModelDF['Data'].max(), gridFt=[False,False], gridM=[False,False], t=titleText, primaryUnit='m', cBarFormat ='%3.0f',cBarLabel ='Resistivity (ohm-m)', showPoints=False, norm=0, whichTicks='major', reverse=False):
  plt.rcParams["figure.dpi"] = 300  
  plt.rcParams['xtick.bottom'] = plt.rcParams['xtick.labelbottom'] = False
  plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = plt.rcParams["xtick.top"] = True
  plt.sca(ax)
 

  vmax90 = np.percentile(v, 90)
  vmin2 = np.percentile(v, 2)
  vmax = v.max()
  vmin = v.min()
  minx = topoDF['xDist'].min()
  maxx = topoDF['xDist'].max()
  minz = min(z)
  maxz = max(z)

  #xlocsM = ax.get_xticks()
  if maxx>800:
    xlocsM = np.arange(minx,maxx+1,100)
    if max(xlocsM) < maxx:
      xlocsM = np.arange(minx,maxx+101,100)
  else:
    xlocsM = np.arange(minx,maxx+1,50)
    if max(xlocsM) < maxx:
      xlocsM = np.arange(minx,maxx+51,50)

  xlabelsM = [str(int(x)) for x in xlocsM]
  xlocsFt = np.uint16(xlocsM*3.2808399)
  xlabelsFt = [str(int(x)) for x in xlocsFt]


  minFtxLoc = xlocsFt[0]
  maxFtxLoc = xlocsFt[-1]
  xLabelsFtEven = np.arange(np.round(minFtxLoc,-1), np.round(maxFtxLoc,-1), 100)

  if np.round(maxFtxLoc,-1) < maxFtxLoc:
    xLabelsFtEven=np.insert(xLabelsFtEven, len(xLabelsFtEven),np.round(maxFtxLoc,-2))
    xLabelsFtEven=np.insert(xLabelsFtEven, len(xLabelsFtEven),np.round(maxFtxLoc,-2)+100)
  xLocs_FTinM = xLabelsFtEven / 3.2808399

  if np.ceil(maxz)>np.round(maxz,-1):
    zEnd = np.round(maxz,-1)+11
  else:
    zEnd = np.round(maxz,-1)+1
  
  ylocsM = np.arange(np.round(minz,-1),zEnd,10)
  ylabelsM = [str(x) for x in ylocsM]
  yLabelsFt = np.uint16(ylocsM*3.2808399)

  minFtyLabel = yLabelsFt.min()
  maxFtyLabel = yLabelsFt.max()
  yLabelsFtEven = np.arange(0, np.round(maxFtyLabel,-1), 20)
  

  if np.round(maxFtyLabel,-1) < maxFtyLabel:
    yLabelsFtEven=np.insert(yLabelsFtEven, len(yLabelsFtEven),yLabelsFtEven[-1]+20)
  yLocs_FTinM = yLabelsFtEven / 3.2808399

  yLimsM = [ylocsM[1],maxz+3]
  yLimsFt = [np.round(yLimsM[0]*3.2808399, 0), np.round(yLimsM[1]*3.2808399, 0)]

  ax.fill_between(topoDF['xDist'],topoDF['Elevation'],topoDF['Elevation']+10,color='w')
  ax.plot(topoDF['xDist'],topoDF['Elevation'],color='k',linewidth=1)
  ax.scatter(topoDF['xDist'],topoDF['Elevation'],marker='v',edgecolors='w',color='k',s=30)
  if showPoints:
    plt.scatter(x,z, c=v, marker='.', cmap='nipy_spectral', norm=norm)

  xLimsM = [minx,maxx]
  xLimsFt = [np.round(xLimsM[0]*3.2808399, 0), np.round(xLimsM[1]*3.2808399, 0)]
  if reverse:
    xLimsM.reverse()
    xLimsFt.reverse()
    #xlocsM=np.flip(xlocsM)
    #xLocs_FTinM=np.flip(xLocs_FTinM)

  def labelPlot(pUnit, pUnitXLocs, pUnitYLocs, pUnitXLabels,pUnitYLabels, sUnit, sUnitXLocs, sUnitYLocs, sUnitXLabels, sUnitYLabels, xLims=xLimsM, yLims=yLimsM, t=t):
      #matplotlib.rc('font', family='sans-serif')   
      #matplotlib.rc('font', serif='Helvetica') 
      #matplotlib.rc('text', usetex='false')   
      #fontName = {'fontname':'Helvetica'}
      
      #plt.title(t,**fontName)
      ax.set_title(t, fontsize=20)
      ax.set_xlabel('Distance ['+sUnit+']', fontsize = 12)
      ax.set_ylabel('Elevation ['+pUnit+']', fontsize = 14)
      ax.set_xticks(sUnitXLocs)
      ax.set_yticks(pUnitYLocs)
      ax.set_xticklabels(sUnitXLabels,fontsize=12)
      ax.set_yticklabels(pUnitYLabels,fontsize=12)
      ax.set_yticks(pUnitYLocs)
      ax.set_ylim(yLims)
      ax.set_xlim(xLims)
      ax.minorticks_on()

      ax2=ax.twiny()
      ax2.set_xticks(pUnitXLocs)
      ax2.set_xticklabels(pUnitXLabels,fontdict={'fontsize':14})
      ax2.set_xlabel('Distance ['+pUnit+']',fontsize=14)
      ax2.minorticks_on()
      ax2.set_xlim(xLimsM)

      ax3=ax2.twinx()
      ax3.set_yticks(sUnitYLocs)
      ax3.set_yticklabels(sUnitYLabels,fontdict={'fontsize':10})
      ax3.set_ylim(yLims)
      ax3.set_ylabel('Elevation ['+sUnit+']',fontsize=14, rotation=270, labelpad = 20)
      ax3.minorticks_on()
     

      if pUnit=='m':
        if gridM[0]:
          ax2.grid(axis='x',alpha=0.5, c='k', which=whichTicks)
        if gridM[1]:
          ax.grid(axis='y',alpha=0.5, c='k', which=whichTicks)
        if gridFt[0]:
          ax.grid(axis='x',alpha=0.5, c='k', which=whichTicks)
        if gridFt[1]:
          ax3.grid(axis='y',alpha=0.5, c='k', which=whichTicks)
      else:
        if gridFt[0]:
          ax2.grid(axis='x',alpha=0.5, c='k', which=whichTicks)
        if gridFt[1]:
          ax.grid(axis='y',alpha=0.5, c='k', which=whichTicks)
        if gridM[0]:
          ax.grid(axis='x',alpha=0.5, c='k', which=whichTicks)
        if gridM[1]:
          ax3.grid(axis='y',alpha=0.5, c='k', which=whichTicks)

  if primaryUnit in ['m', 'meters', 'meter', 'metres', 'metre', 'metric']:
    pUnit = 'm'
    pUnitXLocs = xlocsM
    pUnitYLocs = ylocsM
    pUnitXLabels = xlabelsM
    pUnitYLabels = ylabelsM

    sUnit = 'ft'
    sUnitXLocs = xLocs_FTinM
    sUnitYLocs = yLocs_FTinM
    sUnitXLabels = xLabelsFtEven
    sUnitYLabels = yLabelsFtEven
    labelPlot(primaryUnit, pUnitXLocs, pUnitYLocs, pUnitXLabels,pUnitYLabels, sUnit, sUnitXLocs, sUnitYLocs, sUnitXLabels, sUnitYLabels)
  
  elif primaryUnit in ['f', 'ft', 'feet', 'foot', 'US']:
    pUnit = 'ft'
    pUnitXLocs = xLocs_FTinM
    pUnitYLocs = yLocs_FTinM
    pUnitXLabels = xLabelsFtEven
    pUnitYLabels = yLabelsFtEven
    
    sUnit = 'm'
    sUnitXLocs = xlocsM
    sUnitYLocs = ylocsM
    sUnitXLabels = xlabelsM
    sUnitYLabels = ylabelsM
    labelPlot(primaryUnit, pUnitXLocs, pUnitYLocs, pUnitXLabels,pUnitYLabels, sUnit, sUnitXLocs, sUnitYLocs, sUnitXLabels, sUnitYLabels)  
  
  else:
    pUnit = 'm'
    pUnitXLocs = xlocsM
    pUnitYLocs = ylocsM
    pUnitXLabels = xlabelsM
    pUnitYLabels = ylabelsM

    sUnit = 'ft'
    sUnitXLocs = xLocs_FTinM
    sUnitYLocs = yLocs_FTinM
    sUnitXLabels = xLabelsFtEven
    sUnitYLabels = yLabelsFtEven
    labelPlot(primaryUnit, pUnitXLocs, pUnitYLocs, pUnitXLabels,pUnitYLabels, sUnit, sUnitXLocs, sUnitYLocs, sUnitXLabels, sUnitYLabels)

  cbar = fig.colorbar(im, ax=ax,orientation='horizontal', aspect=50, extend='both',ticks=cbarTicks,format=cBarFormat)
  cbar.ax.tick_params(labelsize=16)#set_ticks(cbarTicks)
  cbar.set_label(label=cBarLabel, size=20)

  ax_h, ax_w = ax.bbox.height, ax.bbox.width
  axRatio = ax_w/ax_h
  aspRatio = (xLimsM[1]-xLimsM[0])/(yLimsM[1]-yLimsM[0])
  vertExag = abs(round(aspRatio/axRatio, 1))
  ax.annotate('Vert.Exag: '+str(vertExag)+'x',xy=(0.95,0.05),xycoords='axes fraction')
  #fig.tight_layout()
  fig.set_facecolor("w")

  plt.show(fig)
  return fig, ax
  #fig.tight_layout()

  

In [ ]:
maxXDist = max(np.float_(electrodes))

def makeColormesh(x,z,v,colMap='nipy_spectral', cBarFormat ='%3.0f', cBarLabel ='Resistivity (ohm-m)', cMin=resistModelDF['Data'].min(),cMax=resistModelDF['Data'].max(), griddedFt=[False,False], griddedM=[False,False], title=invFilePath.stem,normType='linear', primaryUnit='m', showPoints=False,whichTicks='major', reverse=False):#, pe,xOmit,zOmit):
  for i in enumerate(v):
      v[i[0]] = abs(float(i[1]))

  #v=v[(v<=cMax)&(v>=cMin)].copy()
  #indVals = v.index.copy()
  #v.reset_index(drop=True,inplace=True)
  #print(x)
  #print(indVals)
  #print(z)

  #x=x[indVals].reset_index(drop=True,inplace=True)
  #z=z[indVals].reset_index(drop=True,inplace=True)

  xi, zi = np.linspace(min(x), max(x), int(max(x))), np.linspace(min(z), max(z), int(max(z)))
  xi, zi = np.meshgrid(xi, zi)

  vi = scipy.interpolate.griddata((x, z), v, (xi, zi))#, method='linear')

  ptSize = round(100 / maxXDist * 35, 1)

  fig, axes = plt.subplots(1)
  cmap = matplotlib.cm.binary
  my_cmap = cmap(np.arange(cmap.N))
  my_cmap[:,-1] = np.linspace(0,1,cmap.N)
  my_cmap = matplotlib.colors.ListedColormap(my_cmap)

  vmax98 = np.percentile(v, 98)
  vmin2 = np.percentile(v, 2)
  minx = min(x)
  maxx = max(x)
  minz = min(z)
  maxz = max(z)

  vmax = cMax
  vmin = cMin

  #if cMax >= resistModelDF['Data'].max():
  #  vmax = vmax98
  #else:
  #  vmax = cMax
  #if cMin <= resistModelDF['Data'].min():
  #  vmin = vmin2
  #else:
  #  vmin = cMin
  #cbarTicks = np.arange(np.round(vmin,-1),np.round(vmax-1)+1,10) 

  if normType=='log':
    if vmin <= 0:
      vmin = 0.1
    norm = matplotlib.colors.LogNorm(vmin = vmin, vmax = vmax)
    #cBarFormat = '%.1e'
    #cbarTicks = np.logspace(np.log10(vmin),np.log10(vmax),num=10)
  else:
    norm = matplotlib.colors.Normalize(vmin = vmin, vmax = vmax)
  
  arStep = np.round((vmax-vmin)/10,-1)

  cbarTicks = np.arange(np.round(vmin, -1), np.ceil(vmax/10)*10,arStep)


  #im = self.axes.imshow(vi, vmin=vmin, vmax=vmax, origin='lower',
  im = axes.imshow(vi, origin='lower',
              extent=[minx, maxx, minz, maxz],
              aspect='auto',
              cmap=colMap,
              norm = norm,
              interpolation='spline36')

  
  f, a = plotPretty(x,z,v,fig=fig,im=im,ax=axes,colMap=colMap,cMin=cMin,cMax=cMax, gridM=griddedM, gridFt=griddedFt, primaryUnit=primaryUnit,cbarTicks=cbarTicks,cBarFormat=cBarFormat,showPoints=showPoints, norm=norm, whichTicks=whichTicks, reverse=reverse)

  plt.close(f)
  return f, a

In [ ]:
x = resistModelDF['x'].copy()
z = resistModelDF['zElev'].copy()
v = resistModelDF['Data'].copy()
plt.rcParams['figure.dpi'] = 220
plt.rcParams['figure.figsize'] = (35,10)

[figure, axis] = makeColormesh(x,z,v,colMap='nipy_spectral', griddedM=[False,False],griddedFt=[False,True],normType='log', primaryUnit='ft', cMin=16,cMax=500,showPoints=False, whichTicks='major', reverse=True)